<a href="https://colab.research.google.com/github/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Maria_Amado_e_Fernanda_Lisboa_report_handson_3_jupyter_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on 3: Portable Parallel Programming with MPI

M. Amado$^1$, F. Lisboa$^1$

$^1$ Department of Computer Engenier – University SENAI CIMATEC, Salvador, Bahia, Brazil  

# Abstract

A paralelização da execução de tarefas, como operações matemáticas, tem sido aplicadas no dia-a-dia. Há algumas formas de aplicar esse mecanismo em código, seja por memória compartilhada ou distribuída. O presente trabalho busca aplicar os conceitos de memória distribuída, utilizando MPI (Message Passing Information) em algoritmos que realizem operações algébricas e matemáticas. Isso é feito utilizando as principais funções dessa biblioteca (_send_ e _receive_), onde é possível perceber que com uma maior quantidade de dados e valores de entrada a paralelização é mais eficiente na execução de operações como o cálculo: operações matemáticas simples; funções polinomiais; e, diagonal principal, subdiagonal e superdiagonal. Sendo assim, é visualizada a importância da utilização desse mecanismo, mesmo em cálculos do cotidiano.

# Introduction

A paralelização na computação se refere a múltiplas instruções e tipos de dados e envolve decomposição do domínio como um modo de particionar da carga de trabalho [1]. Há diversas operações matemáticas que possuem alto grau de paralelismo [1], aplicadas no dia-a-dia. Em computadores comuns, sem aplicação de alguma técnica de paralelismo, a medida que os valores dessas operações matemáticas vão crescendo, aumenta-se o tempo de execução desses algoritmos, já que é feito de maneira sequencial. A paralelização surge como um método para reduzir esse tempo de execução, sobretudo para valores e volumes de dados mais altos. Esse procedimento lida com partículas de mudanças em ambos domínio e competência de processadores [2], sendo uma das principais bases  a abordagem "dividir para conquistar" [1].


A comunicação entre os processos pode ser feita através de mensagens através de memória compartilhada ou distribuída. Dentro desse cenário surge o MPI (Messsage Passing Interface), o qual diversos processos paralelos trabalham concorrentemente em busca de um objetivo comum utilizando "mensagens" entre si [1]. O MPI possui uma coleção de funções, sendo suas principais de envio e recebimento de informações entre os processos. 

O presente trabalho tem como objetivo otimizar a execução de algoritmos de operações algébricas e matemátricas utilizando conceitos de memória distribuída. Assim, são realizados alguns problemas abordados nessas áreas, utilizando a biblioteca do MPI para aplicar esse paradigma de paralelização como uma forma de otimizar o tempo de execução dos algoritmos. Os problemas são dividos nas seguintes sessões: *basic operations*, abordando a paralelização para operações básicas como soma, subtração e múltiplicação de elementos de um *array*; *algebraic function*, trazendo o cálculo de uma função polinomial de 3º grau para dado os valores de seus coeficientes e um *x* em questão; e, *trigiagonal matrix*, abordando a soma dos valores da diagonal principal, subdiagonal e superdiagonal de uma matriz quadrada.

# Results and Discussion

### Instalando o MPI:

In [1]:
from IPython.display import clear_output

In [2]:
!sudo apt-get install openmpi-bin
clear_output(wait=False)

### Session 1: Basic Operations

Nessa sessão trabalhamos com a paralelização utilizando conceitos de memória distribuída para operações básicas como adição, subtração e multiplicação em um array.

A partir de uma função generalizada, obtemos o valor resultante da operação. Além disso, essa modificação promoveu apenas um envio, ao invés de dois (operação e valor), e decidiu-se utilizar o id do processo como identificador da operação que seria realizada. A lógica utilizada nessa implementação foi replicada nas demais sessões, sempre enviando os dados em forma de array ou matriz e associando os processos a posição do array correspondente.

Código sequencial.

In [4]:
%%writefile op.c
#include <stdio.h>
#include <mpi.h>
#define SIZE 12

int getOperationResult (char operation, int array[SIZE])
{
  int value;
  switch (operation)
  {
  case '+':
    value = 0;
    for (int i = 0; i < SIZE; i++)
      value += array[i];
    break;
  case '-':
    value = 0;
    for (int i = 0; i < SIZE; i++)
      value -= array[i];
    break;
  case '*':
    value = 1;
    for (int i = 0; i < SIZE; i++)
      value *= array[i];
    break;
  }
  return value;
}

int main(int argc, char **argv)
{
  int i, sum = 0, subtraction = 0, mult = 1, result, value;
  int array[SIZE];
  char ops[] = {'+', '-', '*'};
  char operationsRec;
  int numberProcess, id, to, from, tag = 1000;

  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
  MPI_Status status;

  if (id == 0)
  {
    for (i = 0; i < SIZE; i++)
    {
      array[i] = i + 1;
      printf("%d %d\t", i, array[i]);
    }
    printf("\n");
    for (to = 1; to < numberProcess; to++)
    {
      MPI_Send(&array, SIZE, MPI_INT, to, tag, MPI_COMM_WORLD);
      MPI_Send(&ops[to - 1], 1, MPI_CHAR, to, tag, MPI_COMM_WORLD);
    }

    for (to = 1; to < numberProcess; to++)
    {
      MPI_Recv(&result, 1, MPI_INT, to, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&operationsRec, 1, MPI_CHAR, to, tag, MPI_COMM_WORLD, &status);
      printf("(%c) = %d\n", operationsRec, result);
    }
  }
  else
  {
    MPI_Recv(&array, SIZE, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);
    MPI_Recv(&operationsRec, 1, MPI_CHAR, 0, tag, MPI_COMM_WORLD, &status);

    value = getOperationResult(operationsRec, array);

    MPI_Send(&value, 1, MPI_INT, 0, tag, MPI_COMM_WORLD);
    MPI_Send(&operationsRec, 1, MPI_CHAR, 0, tag, MPI_COMM_WORLD);
  }

  MPI_Finalize();
  return 0;
}

Writing op.c


In [5]:
!mpicc op.c -o obj
!mpirun --allow-run-as-root --np 4 ./obj 

0 1	1 2	2 3	3 4	4 5	5 6	6 7	7 8	8 9	9 10	10 11	11 12	
(+) = 78
(-) = -78
(*) = 479001600


Código com a utilização de paralelização usando memória distribuída.

In [6]:
%%writefile basic_opt.c
#include <stdio.h>
#include <mpi.h>
#define SIZE 12

int getOperationResult(int operation, int array[SIZE])
{
    int value;
    switch (operation)
    {
    case 1:
        value = 0;
        for (int i = 0; i < SIZE; i++)
            value += array[i];
        break;
    case 2:
        value = 0;
        for (int i = 0; i < SIZE; i++)
            value -= array[i];
        break;
    case 3:
        value = 1;
        for (int i = 0; i < SIZE; i++)
            value *= array[i];
        break;
    }
    return value;
}

char getOperation(int id)
{
    switch (id)
    {
    case 1:
        return '+';
    case 2:
        return '-';
    case 3:
        return '*';
    default:
        break;
    }
}

int main(int argc, char **argv)
{
    int i, sum = 0, subtraction = 0, mult = 1, result, value;
    int array[SIZE];
    int numberProcess, id, to, from, tag = 1000;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
    MPI_Status status;

    if (id == 0)
    {
        for (i = 0; i < SIZE; i++)
        {
            array[i] = i + 1;
            printf("%d %d\t", i, array[i]);
        }
        printf("\n");
        for (to = 1; to < numberProcess; to++)
        {
            MPI_Send(&array, SIZE, MPI_INT, to, tag, MPI_COMM_WORLD);
        }

        for (from = 1; from < numberProcess; from++)
        {
            MPI_Recv(&result, 1, MPI_INT, from, tag, MPI_COMM_WORLD, &status);
            char operation = getOperation(from);
            printf("(%c) = %d\n", operation, result);
        }
    }
    else
    {
        MPI_Recv(&array, SIZE, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);

        for (int j = 1; j < numberProcess; j++)
        {
            if (id == j)
            {
                value = getOperationResult(id, array);
            }
        }

        MPI_Send(&value, 1, MPI_INT, 0, tag, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}

Writing basic_opt.c


In [7]:
!mpicc basic_opt.c -o obj
!mpirun --allow-run-as-root --np 4 ./obj 

0 1	1 2	2 3	3 4	4 5	5 6	6 7	7 8	8 9	9 10	10 11	11 12	
(+) = 78
(-) = -78
(*) = 479001600


### Session 2: Algebraic Function

Já nessa sessão, a paralelização é feita para funções polinomiais.

Código sequencial.

In [8]:
%%writefile obj.c

#include <stdio.h>

int main (int argc, char **argv){

  double coefficient[4], total, x;
  char c;

  printf ("\nf(x) = a*x^3 + b*x^2 + c*x + d\n");

  for(c = 'a'; c < 'e'; c++) {
    printf ("\nEnter the value of the 'constants' %c:\n", c);
    scanf ("%lf", &coefficient[c - 'a']);
  }

  printf("\nf(x) = %lf*x^3 + %lf*x^2 + %lf*x + %lf\n", coefficient[0], coefficient[1], coefficient[2], coefficient[3]);

  printf("\nEnter the value of 'x':\n");
  scanf("%lf", &x);

  total = (coefficient[0]* x * x * x) + (coefficient[1]* x * x) + (coefficient[2]* x + coefficient[3]);

  printf("\nf(%lf) = %lf*x^3 + %lf*x^2 + %lf*x + %lf = %lf\n", x, coefficient[0], coefficient[1], coefficient[2], coefficient[3], total);

  return 0;
}

Writing obj.c


In [ ]:
!gcc obj.c -o obj
! ./obj


f(x) = a*x^3 + b*x^2 + c*x + d

Enter the value of the 'constants' a:


Código com a utilização de paralelização usando memória distribuída.

In [ ]:
%%writefile algebraic.c
#include <stdio.h>
#include <mpi.h>
#define SIZE 5

double getOperationResult(int processIndex, double array[SIZE])
{
  double term;
  double x = array[0];
  switch (processIndex)
  {
  case 1:
    term = (x * x * x) * array[1];
    break;
  case 2:
    term = (x * x) * array[2];
    break;
  case 3:
    term = x * array[3] + array[4];
    break;
  }
  return term;
}

int main(int argc, char **argv)
{
    int i;
    double result, value, x, total = 0;
    double array[SIZE];
    int numberProcess, id, to, from, tag = 1000;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
    MPI_Status status;

    if (id == 0)
    {
        char c;
        array[1] = 40; //a
        array[2] = 20; //b
        array[3] = 10; //c
        array[4] = 1;  //d

        printf("\nf(x)=%lf*x^3+%lf*x^2+%lf*x+%lf\n", array[1], array[2], array[3], array[4]);
        //printf("\nEnter the value of ’x’:\n");
        //scanf("%lf", &x);
        array[0] = 2; //x

        for (to = 1; to < numberProcess; to++)
        {
            MPI_Send(&array, SIZE, MPI_DOUBLE, to, tag, MPI_COMM_WORLD);
        }

        for (from = 1; from < numberProcess; from++)
        {
            MPI_Recv(&result, 1, MPI_DOUBLE, from, tag, MPI_COMM_WORLD, &status);
            total = total + result;
        }
        printf("\nf(%lf) = %lf*x^3 + %lf*x^2 + %lf*x + %lf = %lf\n", array[0], array[1], array[2], array[3], array[4], total);
    }
    else
    {
        MPI_Recv(&array, SIZE, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD, &status);

        for (int j = 1; j < numberProcess; j++)
        {
            if (id == j)
            {
                value = getOperationResult(id, array);
            }
        }

        MPI_Send(&value, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc algebraic.c -o ob
!mpirun --allow-run-as-root --np 4 ./ob 

### Session 3: Tridiagonal Matrix

Aqui apresentamos a solução para a adição de valores da diagonal principal, superdiagonal e subdiagonal de uma matriz.


Código sequencial.

In [ ]:
%%writefile matrix.c

#include <stdio.h>
#define ORDER 4

void printMatrix (int m[][ORDER]) {
  int i, j;
  for(i = 0; i < ORDER; i++) {
    printf ("| ");
    for (j = 0; j < ORDER; j++) {
      printf ("%3d ", m[i][j]);
    }
    printf ("|\n");
  }
  printf ("\n");
}

int main (int argc, char **argv){

  int k[3] = {100, 200, 300};
  int matrix[ORDER][ORDER];
  int i, j;

  for(i = 0; i < ORDER; i++) {
    for(j = 0; j < ORDER; j++) {
      if( i == j )
        matrix[i][j] = i + j +1;
      else if(i == (j + 1)) {
        matrix[i][j] = i +  j + 1;
        matrix[j][i] = matrix[i][j];
      } else
           matrix[i][j] = 0;
     }
  }

  printMatrix(matrix);

  for(i = 0; i < ORDER; i++){
       matrix[i][i]     += k[0];  //main diagonal
     matrix[i + 1][i] += k[1];    //subdiagonal
     matrix[i][i + 1] += k[2];    //superdiagonal
  }

  printMatrix(matrix);

  return 0;
}

In [ ]:
!gcc matrix.c -o matrix
!./matrix

Código com a utilização de paralelização usando memória distribuída.

In [ ]:
%%writefile matrix.c
#include <stdio.h>
#include <mpi.h>
#define ORDER 4

int matrix[ORDER][ORDER];
int ma[ORDER][ORDER]; //matriz auxiliar
int k[3] = {100, 200, 300};

void printMatrix (int m[][ORDER]) {
    int i, j;
    for(i = 0; i < ORDER; i++) {
    printf ("| ");
    for (j = 0; j < ORDER; j++) {
    printf ("%3d ", m[i][j]);
    }
    printf ("|\n");
    }
    printf ("\n");
}

void getOperationResult(int operation, int m[][ORDER])
{
    int i;
    switch (operation)
    {
    case 1:
        for(i = 0; i < ORDER; i++){
          ma[i][i] = m[i][i] + k[0]; //main diagonal
        }
        break;
    case 2:
        for(i = 0; i < ORDER; i++){
          ma[i + 1][i] = m[i + 1][i] + k[1]; //subdiagonal
        }
        break;
    case 3:
        for(i = 0; i < ORDER; i++){
          ma[i][i + 1] = m[i][i + 1] + k[2]; //superdiagonal
        }
        break;
    }
 }

 void buildMatrix(int operation, int m[][ORDER])
{
  int i;
    switch (operation)
    {
    case 1:
        for(i = 0; i < ORDER; i++){
          matrix[i][i] = m[i][i];
        }
        break;
    case 2:
        for(i = 0; i < ORDER; i++){
          matrix[i + 1][i] = m[i + 1][i];
        }
        break;
    case 3:
        for(i = 0; i < ORDER; i++){
          matrix[i][i + 1] = m[i][i + 1];
        }
        break;
    }
 }

int main(int argc, char **argv)
{
    int i, result[ORDER][ORDER], matrixInit[ORDER][ORDER];
    int numberProcess, id, to, from, tag = 1000;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
    MPI_Status status;

    if (id == 0)
    {
          int i, j;
          for(i = 0; i < ORDER; i++) {
            for(j = 0; j < ORDER; j++) {
            if( i == j )
            matrixInit[i][j] = i + j +1;
            else if(i == (j + 1)) {
            matrixInit[i][j] = i + j + 1;
            matrixInit[j][i] = matrixInit[i][j];
            } else
            matrixInit[i][j] = 0;
            }
          }
          printMatrix(matrixInit);
        for (to = 1; to < numberProcess; to++)
        {
            MPI_Send(&matrixInit, ORDER*ORDER, MPI_INT, to, tag, MPI_COMM_WORLD);
        }

        for (from = 1; from < numberProcess; from++)
        {
            MPI_Recv(&result, ORDER*ORDER, MPI_INT, from, tag, MPI_COMM_WORLD, &status);
            buildMatrix(from, result);
        }
        printMatrix(matrix);
    }
    else
    {
        int matrixSent[ORDER][ORDER];
        
        MPI_Recv(&matrixSent, ORDER*ORDER, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);
        
        for (int j = 1; j < numberProcess; j++)
        {
            if (id == j)
            {
                getOperationResult(id, matrixSent);
            }
        }

        MPI_Send(&ma, ORDER*ORDER, MPI_INT, 0, tag, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}

In [ ]:
!mpicc matrix.c -o matrix
!mpirun --allow-run-as-root --np 4 ./matrix 

# Conclusions

Com a prática realizada, foi possível observar o impacto que a paralelização dos processos tem sobre um algoritmo de alto custo computacional, como a multiplicação de matrizes. Também observamos a importância de utilizar o melhor número de processos para execução do código a fim de reduzir o tempo de execução, e a instabilidade.

# References

[1] Karniadakis, G., & Kirby II, R. (2003). Parallel Scientific Computing in C and MPI: A Seamless Approach to Parallel Algorithms and their Implementation. Cambridge: Cambridge University Press. doi:10.1017/CBO9780511812583

[2] Alessandra Monteleone, Gaetano Burriesci, Enrico Napoli. A distributed-memory MPI parallelization scheme for multi-domain incompressible SPH. Journal of Parallel and Distributed Computing. v. 170, 2022. pp. 53-67, doi:
10.1016/j.jpdc.2022.08.004.

